# Project 1

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, tree

In [3]:
dataset = pd.read_csv('train.csv')
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
dataset.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
dataset['Age'].mean()

29.69911764705882

In [6]:
age_average_replace = np.where(dataset['Age'].isnull(),29,dataset['Age'])

In [7]:
dataset['Age'] = age_average_replace

In [8]:
label_encoder = preprocessing.LabelEncoder()

In [9]:
encoded_sex=label_encoder.fit_transform(dataset['Sex'])

In [10]:
predictors=pd.DataFrame([encoded_sex, dataset['Age'], dataset['Fare']]).T

In [11]:
tree_model = tree.DecisionTreeClassifier(max_depth=6)

In [12]:
tree_model.fit(X=predictors, y=dataset['Survived'])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [13]:
with open('ProjectOne.dot', 'w') as f:
    f=tree.export_graphviz(tree_model,feature_names=['Sex', 'Age','Fare'],out_file=f);

In [14]:
# testing for accuracy of the model
print('Accuracy of algorithm: ')
tree_model.score(X=predictors,y=dataset['Survived'])

Accuracy of algorithm: 


0.8271604938271605

## 2. PREDICTION

In [15]:
dataset_test = pd.read_csv('test.csv')
dataset_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S


In [17]:
dataset_test['Age'].mean()

33.23741007194245

In [18]:
new_age_things = np.where(dataset_test['Age'].isnull(),33,dataset_test['Age'])

In [19]:
dataset_test['Age'] = new_age_things

In [20]:
encoded_sex_test = label_encoder.fit_transform(dataset_test['Sex'])

In [21]:
test_features = pd.DataFrame([encoded_sex_test,dataset_test['Age'],dataset_test['Fare']]).T

In [22]:
predictor_test = tree_model.predict(X=test_features)

In [23]:
predicted_output = pd.DataFrame({'PassengerId':dataset_test['PassengerId'], 'Survived':predictor_test})

In [24]:
predicted_output.to_csv('Output1.csv', index=False);

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
dataset.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [27]:
label_encoder = preprocessing.LabelEncoder()

In [28]:
dataset['Sex'] = label_encoder.fit_transform(dataset['Sex'])

In [29]:
randomforest_model = RandomForestClassifier(n_estimators=1000, max_features=2, oob_score = True)

In [30]:
features = ['Sex', 'Fare',  'Age']

In [31]:
randomforest_model.fit(X=dataset[features], y=dataset['Survived'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [32]:
print('OOB ACCURACY: ')
print(randomforest_model.oob_score)

OOB ACCURACY: 
True


In [33]:
print('OOB ACCURACY: ')
print(randomforest_model.oob_score_)

OOB ACCURACY: 
0.7968574635241302


In [34]:
for features,imp in zip(features, randomforest_model.feature_importances_):
    print(features,imp);

Sex 0.28349684199396114
Fare 0.4271267096054431
Age 0.28937644840059584


from the above, 'Sex, Fare, Age' are the important variable for to be use in the algorithm with the 'Fare' having the highest value.